In [1]:
import requests
import os
import numpy as np
from netCDF4 import Dataset
import datetime as dt
import shutil
import importlib as imp
import scipy.io
import gsw

In [2]:
def qc_full_profile(N_PROF,new_var,qc_val):
    if N_PROF == 1:
        new_var[:]=qc_val
    elif N_PROF == 2:
        new_var[0,:]=qc_val
        qc_to_consider=new_var[1,:].tobytes().decode().strip()
        i_pres_filled=[pos for pos, char in enumerate(qc_to_consider) if char in ['1','2','3','4','5','8']]
        new_var[1,i_pres_filled]=qc_val
    else:
        print("WARNING: this number of profile: ",N_PROF,"is not handled by this code, yet")

In [5]:
# use an other NEMO floats profile file to create a format 3.1 template
owc_dir="C:/Users/ddobler/Documents/08_DD_scripts/06_OWC_matlab/"
l_dmqc_operator="PRIMARY | https://orcid.org/0000-0002-8947-2922 | Delphine DOBLER, Euro-Argo ERIC"

dac='coriolis'

l_wmos=['3902007','3902009','3902010','3902011']
l_apply_OWC_lst=['Y','Y','N','N']
force_l_psal_error_lst=['N','N','Y','Y']
l_psal_error=[0,0,0.02,0.02] # if uncertainty are too high, not suitable for argo ref database err > 0.015

l_wmos=['3902012']
l_apply_OWC_lst=['N']
force_l_psal_error_lst=['N']
l_psal_error=[0] # if uncertainty are too high, not suitable for argo ref database err > 0.015

l_wmos=['3902013']
l_apply_OWC_lst=['N']
force_l_psal_error_lst=['N']
l_psal_error=[0] # if uncertainty are too high, not suitable for argo ref database err > 0.015

l_wmos=['3902014']
l_apply_OWC_lst=['N']
force_l_psal_error_lst=['N']
l_psal_error=[0] # if uncertainty are too high, not suitable for argo ref database err > 0.015

l_psal_accuracy=0.01
l_temp_accuracy=0.002
l_pres_accuracy=2.4


for iwmo in range(len(l_wmos)):
#for iwmo in range(1):
    
    wmo=l_wmos[iwmo]
    l_apply_OWC=l_apply_OWC_lst[iwmo]
    force_l_psal_error=force_l_psal_error_lst[iwmo]
    

    print("\n########################################################")
    print("STEP 0: A few intialisations\n")
    

    date_courante=dt.datetime.now(dt.timezone.utc)
    date_courante_str2=date_courante.strftime("%Y%m%d%H%M%S")
    date_courante_str=date_courante.strftime("%Y-%m-%dT%H:%M:%SZ")
    
    
    l_data_mode='D'
    
    
    l_calib_date=date_courante_str2
    l_calib_eq_pres='none'
    l_calib_eq_temp='none'
    l_calib_comment_pres='The quoted error is manufacturer specified accuracy in dbar.'
    l_calib_comment_temp='The quoted error is manufacturer specified accuracy with respect to ITS-90 at time of laboratory calibration.'
    l_calib_coef_pres='none'
    l_calib_coef_temp='none'
    l_calib_eq_ic_param='not applicable'
    l_calib_comment_ic_param='not applicable'
    l_calib_coef_ic_param='not applicable'

    l_calib_coef_psal_KO='none'
    l_calib_eq_psal_KO='none'
    l_calib_comment_psal_KO='Salinity data are bad and unadjustable.' # will be triggered in the section for specific wmo/cycle QC4 setting
    
    if l_apply_OWC == 'N':
        l_calib_coef_psal_OK='none' 
        l_calib_eq_psal_OK='none'
        l_calib_comment_psal_OK='No significant salinity drift detected. The quoted error is max[0.01, adjustment uncertainty] in PSS-78.'

    if l_apply_OWC == 'Y':
        l_calib_eq_psal_OK='PSAL_ADJUSTED = gsw.SP_from_C(COND_ADJUSTED,TEMP_ADJUSTED,PRES_ADJUSTED); COND_ADJUSTED = pcond_factor * COND; COND = gsw.C_from_SP(PSAL,TEMP_ADJUSTED,PRES_ADJUSTED)'
        l_calib_comment_psal_OK='Small bias and drift. OW least squares fit adopted. Error = maximum [statistical uncertainty, 0.01] in PSS-78.' + \
                         'OWC Method, 3.0, CTD2024V01 & ARGO2023V03, use_pres_gt=1200, max_breaks=0.'
        l_calib_coef_psal_OK='' # is different for each cycle, variable instantiated below.
    
    lh_inst       = 'IF'                       # reference table 4: new code to request for Euro-Argo : 'EA'
    lh_step       = 'ARSQ'                     # reference table 12: ARSQ = 'Delayed-mode QC performed'
    lh_sw         = 'OWC'                      # no ref table (string4 format)
    lh_sw_release = '3.0'                      # no ref table (string4 format)
    lh_ref        = 'matlab OWC using ref CTD2024V01 & ARGO2023V03' # no ref table (string64 format)
    lh_date       = date_courante_str2
    lh_action     = 'IP '                      #reference table 7 IP = 'History group for complete input record' as adviced by QC manual.
   
    #Second: QC application
    # see below in section

    #third adjustment:
    calib_dir = owc_dir + "/05_OWC_OUTPUTS/" + wmo + "_final/"
    calib_mat = scipy.io.loadmat(calib_dir + 'cal_'+wmo+'.mat')
    #print(calib_mat.keys())
    calib_cycles=calib_mat["PROFILE_NO"][:]
    calib_SAL=calib_mat["cal_SAL"][:]
    calib_SAL_err=calib_mat["cal_SAL_err"][:]
    pcond_factor=calib_mat["pcond_factor"][:]
    #print(calib_cycles)
    #print(calib_SAL)
    #print(pcond_factor)
        
    wrk_dir=owc_dir + "/06_D_FILES_WRKDIR/" + wmo + "/"

    if not os.path.exists(wrk_dir):
        os.mkdir(wrk_dir)
    if not os.path.exists(wrk_dir + wmo + "_logs_diffs"):
        os.mkdir(wrk_dir + wmo + "_logs_diffs")
    if not os.path.exists(wrk_dir + wmo + "_MODELE"):
        os.mkdir(wrk_dir + wmo + "_MODELE")
    if not os.path.exists(wrk_dir + wmo + "_ORIGINAUX"):
        os.mkdir(wrk_dir + wmo + "_ORIGINAUX")
    if not os.path.exists(wrk_dir + wmo + "_submitted"):
        os.mkdir(wrk_dir + wmo + "_submitted")


    # Retrieve float_serial_no from meta file:
    META_FILE=wrk_dir + wmo + "_MODELE/" + wmo + "_meta.nc"
    if not os.path.exists(META_FILE):
        print("Getting file from data-argo.ifremer.fr")
        URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/"+ wmo+"_meta.nc"
        print(URL)
        # commands
        response = requests.get(URL)
        open(META_FILE, "wb").write(response.content)
    ds_meta = Dataset(META_FILE,'r')
    l_float_serial_no=ds_meta.variables["FLOAT_SERIAL_NO"][:].tobytes().decode().strip()
    l_pi_name=ds_meta.variables["PI_NAME"][:].tobytes().decode().strip()
    print("FLOAT_SERIAL_NUMBER = ",l_float_serial_no)
    ds_meta.close()

    #Retrieve mode and cycle number from prof file
    MPROF_FILE = wrk_dir + wmo + "_MODELE/" + wmo + "_prof.nc"
    if not os.path.exists(MPROF_FILE):
        print("Getting file from data-argo.ifremer.fr")
        URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/"+ wmo+"_prof.nc"
        print(URL)
        # commands
        response = requests.get(URL)
        open(MPROF_FILE, "wb").write(response.content)
    ds_mprof = Dataset(MPROF_FILE,'r')
    CYCLE_NUMBER=ds_mprof.variables["CYCLE_NUMBER"][:]
    print("CYCLE_NUMBER = ",CYCLE_NUMBER)
    DATA_MODE=ds_mprof.variables["DATA_MODE"][:].tobytes().decode().strip()
    print("DATA_MODE = ",DATA_MODE)
    DIRECTION=ds_mprof.variables["DIRECTION"][:].tobytes().decode().strip()
    print("DIRECTION = ",DIRECTION)
    PRES=ds_mprof.variables["PRES"][:]
    PSAL=ds_mprof.variables["PSAL"][:]
    TEMP=ds_mprof.variables["TEMP"][:]
    ds_mprof.close()



    for icycle in range(CYCLE_NUMBER.size):
    #for icycle in range(1,2):
        l_data_mode='D'

        print("\n########################################################")
        print("STEP 1: initiate new D_file from current file\n")
        
        cycle=int(CYCLE_NUMBER[icycle])
        print("for cycle ",cycle)

        # reintialise the comment on psal at each cycle and change for specific cycles in the specific wmo/cycle section if necessary
        l_calib_comment_psal=l_calib_comment_psal_OK
        l_calib_eq_psal=l_calib_eq_psal_OK
        l_calib_coef_psal=l_calib_coef_psal_OK

        old_data_mode=DATA_MODE[icycle]
        lDIRECTION=DIRECTION[icycle]
        if lDIRECTION=='A':
            dir_str=''
        else:
            dir_str='D'
        
        
        IN_FILE=wrk_dir + wmo + "_ORIGINAUX/" + old_data_mode + wmo + "_" + '{:03d}'.format(cycle) + dir_str + "_vORI.nc"
        if not os.path.exists(IN_FILE):
            print("Getting file from data-argo.ifremer.fr")
            URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/profiles/"+old_data_mode+wmo+"_" + '{:03d}'.format(cycle) + dir_str  + ".nc"
            print(URL)
            # commands
            response = requests.get(URL)
            open(IN_FILE, "wb").write(response.content)

        OUT_FILE=wrk_dir + l_data_mode + wmo + "_" + '{:03d}'.format(cycle) + dir_str  + ".nc"

        
        
        print("SRC_FILE=",IN_FILE)
        print("OUT_FILE=",OUT_FILE)
        
        #try: # Create a new NetCDF file

        ds_src = Dataset(IN_FILE,'r')
        #print("ds_src.ncattrs()")
        #print(ds_src.ncattrs())
        ds_src.set_auto_mask(False)
        
        new_ds = Dataset(OUT_FILE, 'w',format='NETCDF3_CLASSIC')
        new_ds.set_auto_mask(False)
        
        na_texte="n/a"

        ini_fmt_version = ds_src.variables['FORMAT_VERSION'][:].tobytes().decode().strip()
        print('Initial format version =',ini_fmt_version)

        # Copy all dimensions and variables
        for dimname, dim in ds_src.dimensions.items():
            if dimname == 'N_HISTORY':
                N_HISTORY=len(dim)
                print('current history record number = ',N_HISTORY)
                new_ds.createDimension(dimname, None)
            elif dimname == 'N_PARAM':
                N_PARAM=len(dim)
                new_ds.createDimension(dimname, N_PARAM)
            elif dimname == 'N_LEVELS':
                N_LEVELS=len(dim)
                new_ds.createDimension(dimname, N_LEVELS)
            elif dimname == 'N_PROF':
                N_PROF=len(dim)
                new_ds.createDimension(dimname, N_PROF)
            elif dimname == 'N_CALIB':
                #N_CALIB=len(dim)
                #if (wmo == '3902013') & (cycle == 30): # Patch for this cycle specifically
                # keep only the last calibration (comment on the added complexity when using this dimension)
                # this may deserve further discussions though.
                N_CALIB=1                
                new_ds.createDimension(dimname, N_CALIB)
            else:
                new_ds.createDimension(dimname, len(dim))

                
        for varname, var in ds_src.variables.items():
            print("\n Treating ", varname)
        
            new_var = new_ds.createVariable(varname, var.dtype, var.dimensions, fill_value=ds_src.variables[varname]._FillValue)
            
            #print("varname, new_var.shape")
            #print(varname, new_var.shape,var.dimensions)
    
            for local_attrs in ds_src.variables[varname].ncattrs():
                if (local_attrs != '_FillValue'):
                    new_var.setncattr(local_attrs,ds_src.variables[varname].getncattr(local_attrs))

            if varname in ['FLOAT_SERIAL_NO','FORMAT_VERSION','PI_NAME','DATE_UPDATE']:
        
                if varname == 'PI_NAME':
                    # keep this one to propagate corrections
                    new_val=l_pi_name
                elif varname == 'FLOAT_SERIAL_NO':
                    # keep this one to propagate corrections
                    old_val=ds_src.variables['FLOAT_SERIAL_NO'][:].tobytes().decode().strip()
                    new_val=l_float_serial_no
                    print("changing FLOAT_SERIAL_NUMBER from ",old_val," to ",new_val)
                elif varname == 'FORMAT_VERSION':
                    new_val = '3.1'
                elif varname == 'DATE_UPDATE':
                    new_val=date_courante_str2
                else:
                    new_val = na_texte
    
                new_array=np.chararray(new_var.shape)
                new_array[:]=" "
                min_size=min(len(new_val),new_array.size)
                #print('min_size=',min_size)
                for i in range(min_size):
                    if len(new_array.shape) == 2:
                        new_array[:,i]=new_val[i]
                    else:
                        new_array[i]=new_val[i]
                    
                new_var[:]=new_array[:]

            # Note from C.Cabanes: for secondary profiles: keep data_mode to R and DATA_STATE_INDICATOR to 2B, no adjustment.
            elif varname in ['DATA_MODE']:
                #specific as only 1 dimension N_prof, not handled above correctly:
                new_var[0]='D'
                if N_PROF > 1:
                    new_var[1:]='R'
                    
            elif varname in ['DATA_STATE_INDICATOR']:
                new_val_1="2C"
                new_val_2="2B"
                new_array=np.chararray(new_var.shape)
                new_array[:]=" "
                min_size_1=min(len(new_val_1),new_array.size)
                min_size_2=min(len(new_val_2),new_array.size)
                #print('min_size=',min_size)
                
                for i in range(min_size_1):
                    new_array[0,i]=new_val_1[i]
                if N_PROF >= 2:
                    for i in range(min_size_2):
                        new_array[1:,i]=new_val_2[i]
                new_var[:]=new_array[:]

            elif varname in ['PARAMETER','SCIENTIFIC_CALIB_EQUATION','SCIENTIFIC_CALIB_COEFFICIENT',
                            'SCIENTIFIC_CALIB_COMMENT','SCIENTIFIC_CALIB_DATE']:
                new_var[:]=ds_src.variables[varname][:,:N_CALIB,:,:]
                

            else:
                #print("last else statement")
                new_var[:]=ds_src.variables[varname][:]
            
            #if new_var[:].dtype == '|S1':
            #    print(varname, " :",new_var[:].tobytes().decode().strip())
            #else:
            #    print(varname, " :",new_var[:])

        
        #Update the global attributes:
        new_ds.title = "Argo float vertical profile"
        new_ds.institution = "CORIOLIS"
        new_ds.source = "Argo float"
        new_ds.history = ""
        new_ds.references = "http://www.argodatamgt.org/Documentation"
        new_ds.user_manual_version = "3.1"
        new_ds.Conventions = "Argo-3.1 CF-1.6"
        new_ds.featureType = "trajectoryProfile"
        new_ds.decoder_version = ds_src.decoder_version
        new_ds.id = "https://doi.org/10.17882/42182"
        new_ds.comment_dmqc_operator = l_dmqc_operator
                
        
        # update the global attribute history:
        tmp=str(ds_src.variables["DATE_CREATION"][:].tobytes(),'utf-8')
        record_date_creation=tmp[:4]+"-"+tmp[4:6]+"-"+tmp[6:8]+"T"+tmp[8:10]+":"+tmp[10:12]+":"+tmp[12:14]+"Z"

        new_ds.history = record_date_creation+ ' creation; '+date_courante_str+' updated'
        print("New history global attr: ",new_ds.history)



        #########################################################################################################
        #########################################################################################################

        print("\n########################################################")
        print("STEP 2: specific QC applications\n")

        for varname in ['PSAL_QC','TEMP_QC','PRES_QC']:
            new_var     = new_ds.variables[varname]
            PRES = new_ds.variables['PRES'][0,:]
            # QC application for 3902007 only for primary profiles
            if (varname =="TEMP_QC") & (wmo =='3902007'):
                qc_full_profile(N_PROF,new_var,'1')
            if (varname =="PSAL_QC") & (wmo =='3902007') & (cycle==1) & (lDIRECTION=='A'):
                new_var[0,:]='1'
                ipresko=np.where((PRES>=70.9) & (PRES <=73.8))[0]
                new_var[0,ipresko]='4'
            if (varname =="PSAL_QC") & (wmo =='3902007') & (cycle==54) & (lDIRECTION=='A'):
                qc_full_profile(N_PROF,new_var,'4')
                l_calib_comment_psal=l_calib_comment_psal_KO
                l_calib_eq_psal=l_calib_eq_psal_KO
                # l_calib_coef_psal for cycle 54 is set in the adjustment section.
            if (varname =="PSAL_QC") & (wmo =='3902007') & (cycle==57) & (lDIRECTION=='A'):
                new_var[0,np.where((PRES>=613) & (PRES <=613))[0]]='4'

            if (varname =="TEMP_QC") & (wmo =='3902009'):
                qc_full_profile(N_PROF,new_var,'1')

            if (varname =="TEMP_QC") & (wmo =='3902010'):
                qc_full_profile(N_PROF,new_var,'1')
            if (varname =="PSAL_QC") & (wmo =='3902010') & ((cycle >=70) | (cycle in [39,40])):
                qc_full_profile(N_PROF,new_var,'4')
                l_calib_comment_psal=l_calib_comment_psal_KO
                l_calib_eq_psal=l_calib_eq_psal_KO

            if (varname =="TEMP_QC") & (wmo =='3902011'):
                qc_full_profile(N_PROF,new_var,'1')
            if (varname =="PSAL_QC") & (wmo =='3902011') & (cycle in [152]):
                qc_full_profile(N_PROF,new_var,'4')
                l_calib_comment_psal=l_calib_comment_psal_KO
                l_calib_eq_psal=l_calib_eq_psal_KO
            if (varname =="PSAL_QC") & (wmo =='3902011') & (cycle==123):
                ipresko=np.where((PRES>=430) & (PRES <=480))[0]
                new_var[0,ipresko]='4'
            if (varname =="PSAL_QC") & (wmo =='3902011') & (cycle==175):
                ipresko=np.where((PRES>=500) & (PRES <=700))[0]
                new_var[0,ipresko]='4'

            if (varname =="PSAL_QC") & (wmo =='3902012') & (cycle==43):
                ipresko=np.where((PRES>=620) & (PRES <=750))[0]
                new_var[0,ipresko]='4'

            if (varname =="PSAL_QC") & (wmo =='3902014') & (cycle==65):
                ipresko=np.where((PRES>=-5) & (PRES <=100))[0]
                new_var[0,ipresko]='4'
                

            # correct QC for secondary if close to primary
            #if (varname in ["PRES_QC","TEMP_QC","PSAL_QC"]) & (N_PROF==2):
            #    print('Treating secondary profile QCs')
            #    qc_to_consider=new_var[1,:].tobytes().decode().strip()
            #    i_ok=[pos for pos, char in enumerate(qc_to_consider) if char in ['1','2','3','5','8']]
            #    if len(i_ok) > 0:
            #        if varname == "TEMP_QC":
            #            varname_raw='TEMP'
            #            lim=0.03
            #        if varname == "PSAL_QC":
            #            varname_raw='PSAL'
            #            lim=0.02
            #        if varname == "PRES_QC":
            #            varname_raw='PRES'
            #            lim=4
            #        var_primary=new_ds.variables[varname_raw][0,:]
            #        var_secondary=new_ds.variables[varname_raw][1,:]
            #        var_primary_ok=var_primary[i_ok]
            #        var_secondary_ok=var_secondary[i_ok]
            #        d_prim_sedonc=np.max(abs(var_primary_ok-var_secondary_ok))
            #        print("max difference between primary and secondary profile values for ",varname_raw," for cycle ",cycle,":",d_prim_sedonc)
            #        if d_prim_sedonc < lim:
            #            print("max difference small enough: qc set to 1 for ",varname_raw," for cycle ",cycle,":",d_prim_sedonc)
            #            new_var[1,i_ok]='1'
            #        else:
            #            print("max difference too large: manual check for ",varname_raw," for cycle ",cycle,":",d_prim_sedonc)


        
        #########################################################################################################
        #########################################################################################################

        print("\n########################################################")
        print("STEP 3: PRES, TEMP and PSAL adjustment\n")
        
        # rules for QCs and fill values consistency will be applied afterwards
        # Note from C.Cabanes: for secondary profiles: keep data_mode to R and DATA_STATE_INDICATOR to 2B, no adjustment.
        
        for varname in ['PRES_ADJUSTED','TEMP_ADJUSTED','PSAL_ADJUSTED']:
            new_var     = new_ds.variables[varname]
            new_var_err = new_ds.variables[varname + '_ERROR'] 
            new_var_qc  = new_ds.variables[varname + '_QC'] 
        
            if varname=='PRES_ADJUSTED':
                new_var[0,:] = new_ds.variables['PRES'][0,:]
                new_var_qc[0,:]=new_ds.variables['PRES_QC'][0,:]
                new_var_err[0,:]=l_pres_accuracy

            if varname=='TEMP_ADJUSTED':
                new_var[0,:] = new_ds.variables['TEMP'][0,:]
                new_var_qc[0,:]=new_ds.variables['TEMP_QC'][0,:]
                new_var_err[0,:]=l_temp_accuracy

            if (varname=='PSAL_ADJUSTED') & (l_apply_OWC=='Y'):
                print("Applying OWC adjustment")
                
                psal_raw=ds_src.variables['PSAL'][0,:]
                temp_raw=new_ds.variables['TEMP'][0,:]
                temp_adj=new_ds.variables['TEMP_ADJUSTED'][0,:]
                pres_raw=new_ds.variables['PRES'][0,:]
                pres_adj=new_ds.variables['PRES_ADJUSTED'][0,:]
                lat=ds_src.variables['LATITUDE'][0]
                lon=ds_src.variables['LONGITUDE'][0]
                
                lpcond_factor=pcond_factor[(calib_cycles == cycle)][0]
                print("lat,lon,lpcond_factor=",lat,lon,lpcond_factor)
                cond_raw = gsw.C_from_SP(psal_raw,temp_adj,pres_adj)
                cond_adj = lpcond_factor * cond_raw
                psal_adj  = gsw.SP_from_C(cond_adj,temp_adj,pres_adj)
    
                lcalib_SAL_err=np.nanmean(calib_SAL_err[:,np.where(calib_cycles==cycle)[0][0]])
                print("lcalib_SAL_err=",lcalib_SAL_err)
                psal_adj_err=max(lcalib_SAL_err,l_psal_accuracy) # as detailed in the QC manual
                print("psal_adj_err=",psal_adj_err)
                if force_l_psal_error== 'Y':
                    psal_adj_err=l_psal_error[iwmo]

                new_var[0,:] = psal_adj[:]
                new_var_qc[0,:]=new_ds.variables['PSAL_QC'][0,:]
                new_var_err[0,:]=psal_adj_err

                l_calib_coef_psal='pcond_factor='+str(lpcond_factor)

                if (wmo =='3902007') & (cycle==54):
                    l_calib_coef_psal=l_calib_coef_psal_KO

            
            if (varname=='PSAL_ADJUSTED') & (l_apply_OWC=='N'):
                print("Not Applying OWC adjustment")
                new_var[0,:] = new_ds.variables['PSAL'][0,:]
                new_var_qc[0,:]=new_ds.variables['PSAL_QC'][0,:]
                if force_l_psal_error== 'Y':
                    psal_adj_err=l_psal_error
                else:
                    psal_adj_err=l_psal_accuracy
                new_var_err[0,:]=psal_adj_err

        #########################################################################################################
        #########################################################################################################

        print("\n########################################################")
        print("STEP 4: Editing CALIB_* fields\n")

        parameter=new_ds.variables['PARAMETER']
        for iparam in range(N_PARAM):
            for iprof in range(1):
                param_name=parameter[iprof,N_CALIB-1,iparam,:].tobytes().decode().strip()
                print("Filling in SCIENTIFIC_CALIB_* fields for ",param_name)
                if param_name == 'PRES':
                    for i in range(len(l_calib_eq_pres)):
                        new_ds.variables['SCIENTIFIC_CALIB_EQUATION'][iprof,N_CALIB-1,iparam,i]=l_calib_eq_pres[i]
                    for i in range(len(l_calib_coef_pres)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COEFFICIENT'][iprof,N_CALIB-1,iparam,i]=l_calib_coef_pres[i]
                    for i in range(len(l_calib_comment_pres)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COMMENT'][iprof,N_CALIB-1,iparam,i]=l_calib_comment_pres[i]
                    for i in range(len(l_calib_date)):    
                        new_ds.variables['SCIENTIFIC_CALIB_DATE'][iprof,N_CALIB-1,iparam,i]=l_calib_date[i]
                if param_name == 'TEMP':
                    for i in range(len(l_calib_eq_temp)):
                        new_ds.variables['SCIENTIFIC_CALIB_EQUATION'][iprof,N_CALIB-1,iparam,i]=l_calib_eq_temp[i]
                    for i in range(len(l_calib_coef_temp)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COEFFICIENT'][iprof,N_CALIB-1,iparam,i]=l_calib_coef_temp[i]
                    for i in range(len(l_calib_comment_temp)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COMMENT'][iprof,N_CALIB-1,iparam,i]=l_calib_comment_temp[i]
                    for i in range(len(l_calib_date)):    
                        new_ds.variables['SCIENTIFIC_CALIB_DATE'][iprof,N_CALIB-1,iparam,i]=l_calib_date[i]
                if param_name == 'PSAL':
                    for i in range(len(l_calib_eq_psal)):
                        new_ds.variables['SCIENTIFIC_CALIB_EQUATION'][iprof,N_CALIB-1,iparam,i]=l_calib_eq_psal[i]
                    for i in range(len(l_calib_coef_psal)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COEFFICIENT'][iprof,N_CALIB-1,iparam,i]=l_calib_coef_psal[i]
                    for i in range(len(l_calib_comment_psal)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COMMENT'][iprof,N_CALIB-1,iparam,i]=l_calib_comment_psal[i]
                    for i in range(len(l_calib_date)):    
                        new_ds.variables['SCIENTIFIC_CALIB_DATE'][iprof,N_CALIB-1,iparam,i]=l_calib_date[i]
                if param_name in ['MTIME','NB_SAMPLE_CTD','TEMP_CNDC','PRES_MED','TEMP_MED','PSAL_MED','TEMP_STD','PSAL_STD']:
                    for i in range(len(l_calib_eq_ic_param)):
                        new_ds.variables['SCIENTIFIC_CALIB_EQUATION'][iprof,N_CALIB-1,iparam,i]=l_calib_eq_ic_param[i]
                    for i in range(len(l_calib_coef_ic_param)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COEFFICIENT'][iprof,N_CALIB-1,iparam,i]=l_calib_coef_ic_param[i]
                    for i in range(len(l_calib_comment_ic_param)):    
                        new_ds.variables['SCIENTIFIC_CALIB_COMMENT'][iprof,N_CALIB-1,iparam,i]=l_calib_comment_ic_param[i]
                    for i in range(len(l_calib_date)):    
                        new_ds.variables['SCIENTIFIC_CALIB_DATE'][iprof,N_CALIB-1,iparam,i]=l_calib_date[i]

        for varname in ['SCIENTIFIC_CALIB_EQUATION','SCIENTIFIC_CALIB_COEFFICIENT', \
                       'SCIENTIFIC_CALIB_COMMENT','SCIENTIFIC_CALIB_DATE']:
            val=new_ds.variables[varname][:].tobytes().decode().strip()
            print(varname,':')
            print(val)



        #########################################################################################################
        #########################################################################################################

        print("\n########################################################")
        print("STEP 5: Editing HISTORY_* fields\n")
        for iprof in range(1):
            for i in range(len(lh_inst)): 
                new_ds.variables['HISTORY_INSTITUTION'][N_HISTORY,iprof,i]      = lh_inst[i]
            for i in range(len(lh_step)): 
                new_ds.variables['HISTORY_STEP'][N_HISTORY,iprof,i]             = lh_step[i]
            for i in range(len(lh_sw)): 
                new_ds.variables['HISTORY_SOFTWARE'][N_HISTORY,iprof,i]         = lh_sw[i]
            for i in range(len(lh_sw_release)): 
                new_ds.variables['HISTORY_SOFTWARE_RELEASE'][N_HISTORY,iprof,i] = lh_sw_release[i]
            for i in range(len(lh_ref)): 
                new_ds.variables['HISTORY_REFERENCE'][N_HISTORY,iprof,i]        = lh_ref[i]
            for i in range(len(lh_date)): 
                new_ds.variables['HISTORY_DATE'][N_HISTORY,iprof,i]             = lh_date[i]
            for i in range(len(lh_action)): 
                new_ds.variables['HISTORY_ACTION'][N_HISTORY,iprof,i]           = lh_action[i]
                
        for varname in ['HISTORY_INSTITUTION','HISTORY_STEP','HISTORY_SOFTWARE','HISTORY_SOFTWARE_RELEASE', \
                       'HISTORY_REFERENCE','HISTORY_DATE','HISTORY_ACTION']:
            val=new_ds.variables[varname][:].tobytes().decode().strip()
            print(varname,':')
            print(val)
 

        #########################################################################################################
        #########################################################################################################

        print("\n########################################################")
        print("STEP 6: Applying QC and fill values consistency rules")


        # note: PRES should be the first parameter (test on PRES_QC=4 and apllication on other PARAM_QC)
        
        # Update QCs following new rules 
        # - if no values for RAW, then QC=9 and not ' '; 
        # - if pres_QC=4 then PSAL_QC and TEMP_QC =4;
        # - the same for ADJUSTED counterpart
        # - if  {PARAM}_ADJUSTED_QC = 4 then {PARAM}_ADJUSTED=Fill_value
        # WARNING: This is not tested, nor working for N_PROF > 1.
        i_pres_KO=[]
        i_pres_adj_KO=[]

        # This line is very important, otherwise issue when pres slightly out of boundaries and tests for fill values
        new_ds.set_auto_mask(False)
        
        for varname in ['PRES','TEMP','PSAL']:
        #for varname in ['PRES']:
            
            var_raw         = new_ds.variables[varname][:]
            new_var_raw     = new_ds.variables[varname]
            var_raw_qc      = new_ds.variables[varname + '_QC'][:]
            new_var_raw_qc  = new_ds.variables[varname + '_QC']
            var_adj         = new_ds.variables[varname + '_ADJUSTED'][:]
            new_var_adj     = new_ds.variables[varname + '_ADJUSTED']
            var_adj_qc      = new_ds.variables[varname + '_ADJUSTED_QC'][:]
            new_var_adj_qc  = new_ds.variables[varname + '_ADJUSTED_QC']
            var_adj_err     = new_ds.variables[varname + '_ADJUSTED_ERROR'][:]
            new_var_adj_err = new_ds.variables[varname + '_ADJUSTED_ERROR']
            var_prof_qc     = new_ds.variables['PROFILE_' + varname + '_QC'][:]
            new_var_prof_qc = new_ds.variables['PROFILE_' + varname + '_QC']

            #print('before:',varname)
            #print(new_var_raw[1,:])
            #print('before:',varname + '_QC')
            #print(new_var_raw_qc[1,:].tobytes().decode().strip())
            #print('before:',varname + '_ADJ')
            #print(new_var_adj[1,:])
            #print('before:',varname + '_ADJ_QC')
            #print(new_var_adj_qc[1,:].tobytes().decode().strip())
            #print('before:',varname + '_ERR')
            #print(new_var_adj_err[1,:])  

                
            # if PRES_QC = 4 then the other RAW_QC =4
            if varname == 'PRES':
                #print("selecting PRES set to QC4")
                var_raw_qc_arr=var_raw_qc.tobytes().decode().strip()
                i_raw_pres_KO=[pos for pos, char in enumerate(var_raw_qc_arr) if char == '4']
                var_adj_qc_arr=var_adj_qc.tobytes().decode().strip()
                i_adj_pres_KO=[pos for pos, char in enumerate(var_adj_qc_arr) if char == '4']
            
            if varname in ['TEMP','PSAL']:
                #print("Applying QC4 to TEMP and PSAL for PRES QC4 levels")
                var_raw_qc[0,i_raw_pres_KO]='4'
                new_var_raw_qc[:]=var_raw_qc[:]
                var_adj_qc[0,i_adj_pres_KO]='4'
                new_var_adj_qc[:]=var_adj_qc[:]

            
            for iPROF in range(N_PROF):
                l_data_mode=new_ds.variables["DATA_MODE"][iPROF].tobytes().decode().strip()
                print("iPROF = ",iPROF," and l_data_mode = ",l_data_mode)
                if l_data_mode == "D":
                    #print("if QC = 4 for ADJ (when data_mode = D), then ADJ and ERR = fillValue")
                    var_adj_qc_arr=new_ds.variables[varname + '_ADJUSTED_QC'][iPROF,:].tobytes().decode().strip()
                    i_adj_ko=[pos for pos, char in enumerate(var_adj_qc_arr) if char == '4']
                    #print("i_adj_ko=",i_adj_ko)
                    var_adj[iPROF,i_adj_ko]=99999.0
                    new_var_adj[:]=var_adj[:]
                    var_adj_err[iPROF,i_adj_ko]=99999.0
                    new_var_adj_err[:]=var_adj_err[:]

                
                
                if l_data_mode in ["A","D"]:
                    # if RAW <> FillValue & ADJ = FillValue; RAW_QC = 4 & ADJ_QC = 4 and ERR = FillValue
                    # if RAW = FillValue & ADJ = FillValue; RAW_QC = 9 & ADJ_QC = 9 and ERR = FillValue
                    # if RAW = FillValue; RAW_QC = 9 (Adjusted value can be interpolated sometimes)
                    i_qc4=np.where((var_adj[iPROF,:] >= 99999) & (var_raw[iPROF,:] < 99999))
                    var_adj_qc[iPROF,i_qc4]='4'
                    var_raw_qc[iPROF,i_qc4]='4'
                    var_adj_err[iPROF,i_qc4]=99999.0
                    new_var_adj_qc[:]=var_adj_qc[:]
                    new_var_raw_qc[:]=var_raw_qc[:]
                    new_var_adj_err[:]=var_adj_err[:]

                    # this needs adjustements when there are secondary profiles. 
                    if iPROF == 0:
                        i_qc9=np.where((var_adj[iPROF,:] >= 99999) & (var_raw[iPROF,:] >= 99999))
                        var_adj_qc[iPROF,i_qc9]='9'
                        var_raw_qc[iPROF,i_qc9]='9'
                        var_adj_err[iPROF,i_qc9]=99999.0
                        new_var_adj_qc[:]=var_adj_qc[:]
                        new_var_raw_qc[:]=var_raw_qc[:]
                        new_var_adj_err[:]=var_adj_err[:]
                        

                if iPROF == 0:
                    i_qc9=np.where(var_raw[iPROF,:] >= 99999)
                    var_raw_qc[iPROF,i_qc9]='9'
                    new_var_raw_qc[:]=var_raw_qc[:]

            # Recomputation of profile_QCs:
            new_profile_qc=[None] * N_PROF
            for iPROF in range(N_PROF):
                l_data_mode=new_ds.variables['DATA_MODE'][iPROF].tobytes().decode().strip()
                if l_data_mode in ["A","D"]:
                    qc_to_consider=new_ds.variables[varname + '_ADJUSTED_QC'][iPROF,:]
                else:
                    qc_to_consider=new_ds.variables[varname + '_QC'][iPROF,:]
                    
                qc_to_consider=qc_to_consider.tobytes().decode().strip()
                print(qc_to_consider)
                i_ko=[pos for pos, char in enumerate(qc_to_consider) if char in ['3','4']]
                i_ok=[pos for pos, char in enumerate(qc_to_consider) if char in ['1','2','5','8']]
                n_ko,n_ok=len(i_ko),len(i_ok)
                percent_ok=n_ok/(n_ok+n_ko)
                print('percent_ok=',percent_ok)
                new_profile_qc[iPROF]='A'
                if percent_ok < 1.00 :  new_profile_qc[iPROF]='B'
                if percent_ok < 0.75 :  new_profile_qc[iPROF]='C'
                if percent_ok < 0.50 :  new_profile_qc[iPROF]='D'
                if percent_ok < 0.25 :  new_profile_qc[iPROF]='E'
                if percent_ok == 0   :  new_profile_qc[iPROF]='F'
                    

            new_profile_qc="".join(new_profile_qc)
            old_profile_qc=new_var_prof_qc[:].tobytes().decode().strip()
            print('Modifying the PROFILE_' + varname + '_QC for wmo ',wmo,' cycle ',cycle,' from ', old_profile_qc, ' to ', new_profile_qc)
            new_array=np.chararray(new_var_prof_qc.shape)
            new_array[:]=" "
            for iPROF in range(N_PROF):
                new_array[iPROF]=new_profile_qc[iPROF]
            new_var_prof_qc[:]=new_array[:]

            #print('after:',varname)
            #print(new_var_raw[1,:])
            #print('after:',varname + '_QC')
            #print(new_var_raw_qc[1,:].tobytes().decode().strip())
            #print('after:',varname + '_ADJ')
            #print(new_var_adj[1,:])
            #print('after:',varname + '_ADJ_QC')
            #print(new_var_adj_qc[1,:].tobytes().decode().strip())
            #print('after:',varname + '_ERR')
            #print(new_var_adj_err[1,:])

        ds_src.close()
        new_ds.close()
        
    #except:


########################################################
STEP 0: A few intialisations

FLOAT_SERIAL_NUMBER =  AI2600-21EU006
CYCLE_NUMBER =  [ 1  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74]
DATA_MODE =  RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
DIRECTION =  DAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

########################################################
STEP 1: initiate new D_file from current file

for cycle  1
SRC_FILE= C:/Users/ddobler/Documents/08_DD_scripts/06_OWC_matlab//06_D_FILES_WRKDIR/3902014/3902014_ORIGINAUX/R3902014_001D_vORI.nc
OUT_FILE= C:/Users/ddobler/Documents/08_DD_scripts/06_OWC_matlab//06_D_FILES_WRKDIR/3902014/D3902014_001D.nc
Initial format version = 3.1
current history record number =  3

 Treating  DATA_TYPE

 Tre